In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib as ta
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data1 = pd.read_csv("EURUSD_Candlestick_5_M_BID_31.12.2007-31.12.2012.csv")
data2 = pd.read_csv("EURUSD_Candlestick_5_M_BID_31.12.2012-31.12.2014.csv")
data3 = pd.read_csv("EURUSD_Candlestick_5_M_BID_31.12.2014-19.03.2018.csv")

In [3]:
frames = [data1, data2, data3]
data = pd.concat(frames)
data = data.drop_duplicates()
del data1, data2, data3

In [4]:
data["body"] = (data['Close'] - data ['Open'])*10000
data['upper_tail'] = (data['High'] - data['Open'])*10000
data['lower_tail'] = (data['Close'] - data['Low'])*10000

In [5]:
data.loc[data['body']>0,'upper_tail'] = (data['High'][data['body']>0]-data['Close'][data['body']>0])*10000
data.loc[data['body']>0,'lower_tail'] = (data['Open'][data['body']>0] - data['Low'][data['body']>0])*10000 

In [6]:
data['SMA_50'] = ta.SMA(np.array(data['Close']),timeperiod=50)
data['SMA_20'] = ta.SMA(np.array(data['Close']),timeperiod=20)
data['ATR'] = ta.ATR(np.array(data['High']),np.array(data['Low']),np.array(data['Close']),timeperiod=50)
data['CCI'] = ta.CCI(np.array(data['High']),np.array(data['Low']),np.array(data['Close']),timeperiod=20)
data['SAR']= ta.SAR(np.array(data['High']),np.array(data['Low']))

In [7]:
data = data.set_index(pd.to_datetime(data['Gmt time'],infer_datetime_format=True))
data = data.drop('Gmt time',axis = 1)

In [8]:
data['hour'] = data.index.hour
data['min'] = data.index.minute
data['dayofweek'] = data.index.dayofweek

In [9]:
#JPY
data.loc[:,'JPY']=0
data.loc[data['hour']<9,'JPY'] = 1
#JPY
data.loc[:,'AUD'] = 0
data.loc[data['hour']<6,'AUD'] = 1
data.loc[data['hour']>21,'AUD'] = 1
#EUR
data.loc[:,'EUR'] = 1
data.loc[data['hour']<7,'EUR'] = 0
data.loc[data['hour']>15,'EUR'] = 0
#GBP
data.loc[:,'GBP'] = 1
data.loc[data['hour']<8,'GBP'] = 0
data.loc[data['hour']>16,'GBP'] = 0
#USD
data.loc[:,'USD'] = 1
data.loc[data['hour']<13,'USD'] = 0
data.loc[data['hour']>21,'USD'] = 0

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1074816 entries, 2008-01-01 00:00:00 to 2018-03-20 23:55:00
Data columns (total 21 columns):
Open          1074816 non-null float64
High          1074816 non-null float64
Low           1074816 non-null float64
Close         1074816 non-null float64
Volume        1074816 non-null float64
body          1074816 non-null float64
upper_tail    1074816 non-null float64
lower_tail    1074816 non-null float64
SMA_50        1074767 non-null float64
SMA_20        1074797 non-null float64
ATR           1074766 non-null float64
CCI           1074797 non-null float64
SAR           1074815 non-null float64
hour          1074816 non-null int64
min           1074816 non-null int64
dayofweek     1074816 non-null int64
JPY           1074816 non-null int64
AUD           1074816 non-null int64
EUR           1074816 non-null int64
GBP           1074816 non-null int64
USD           1074816 non-null int64
dtypes: float64(13), int64(8)
memory usage: 180.4 M

In [11]:
for i in np.arange(1,97):
    data['return_'+str(i)] = data['Close'].pct_change(i)
    data['lag_return_'+str(i)] = data['return_1'].shift(i)

In [12]:
# label the return of its next bar
data.loc[:,'up_down'] = 0
data.loc[data['return_1'].shift(-1)>0,'up_down']=1
data = data.drop(columns = ['return_1'])

In [13]:
data = data.dropna(axis=0, how='any')

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1074719 entries, 2008-01-01 08:05:00 to 2018-03-20 23:55:00
Columns: 213 entries, Open to up_down
dtypes: float64(204), int64(9)
memory usage: 1.7 GB


In [15]:
data.head(10)

,Open,High,Low,Close,Volume,body,upper_tail,lower_tail,SMA_50,SMA_20,...,lag_return_92,return_93,lag_return_93,return_94,lag_return_94,return_95,lag_return_95,return_96,lag_return_96,up_down
Gmt time,,,,,,,,,,,,,,,,,,,,,
2008-01-01 08:05:00,1.45865,1.45885,1.45856,1.45873,5004.3375,0.8,1.2,0.9,1.459251,1.459150,...,-0.000089,-0.000528,0.000027,-0.000500,-0.000103,-0.000603,0.000226,-0.000377,0.000933,0
2008-01-01 08:10:00,1.45883,1.45883,1.45847,1.45863,4649.9000,-2.0,0.0,1.6,1.459236,1.459130,...,0.000103,-0.000507,-0.000089,-0.000596,0.000027,-0.000569,-0.000103,-0.000671,0.000226,1
2008-01-01 08:15:00,1.45863,1.45896,1.45845,1.45886,5283.9500,2.3,1.0,1.8,1.459217,1.459097,...,0.000048,-0.000452,0.000103,-0.000349,-0.000089,-0.000439,0.000027,-0.000411,-0.000103,1
2008-01-01 08:20:00,1.45865,1.45905,1.45842,1.45895,5370.3375,3.0,1.0,2.3,1.459204,1.459044,...,-0.000644,-0.000438,0.000048,-0.000391,0.000103,-0.000288,-0.000089,-0.000377,0.000027,0
2008-01-01 08:25:00,1.45902,1.45905,1.45857,1.45887,3738.9375,-1.5,0.3,3.0,1.459187,1.458988,...,-0.000048,0.000151,-0.000644,-0.000493,0.000048,-0.000445,0.000103,-0.000343,-0.000089,1
2008-01-01 08:30:00,1.45883,1.45915,1.45852,1.45913,5427.8375,3.0,0.2,3.1,1.459186,1.458943,...,-0.000514,0.000377,-0.000048,0.000329,-0.000644,-0.000315,0.000048,-0.000267,0.000103,0
2008-01-01 08:35:00,1.45905,1.45922,1.45881,1.45887,4256.5000,-1.8,1.7,0.6,1.459168,1.458883,...,0.000014,0.000713,-0.000514,0.000199,-0.000048,0.000151,-0.000644,-0.000493,0.000048,0
2008-01-01 08:40:00,1.45892,1.45899,1.45848,1.45883,4874.0500,-0.9,0.7,3.5,1.459154,1.458886,...,0.000055,0.000672,0.000014,0.000686,-0.000514,0.000171,-0.000048,0.000123,-0.000644,0
2008-01-01 08:45:00,1.45893,1.45906,1.45861,1.45862,4751.0469,-3.1,1.3,0.1,1.459132,1.458883,...,-0.000117,0.000473,0.000055,0.000528,0.000014,0.000542,-0.000514,0.000027,-0.000048,1


### Split sample by time

In [38]:
cut_at = int(np.floor(len(data)*.66))
train = data.iloc[0:cut_at,:]
test = data.iloc[cut_at:,:]

In [39]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 709314 entries, 2008-01-01 08:05:00 to 2014-09-29 05:30:00
Columns: 213 entries, Open to up_down
dtypes: float64(204), int64(9)
memory usage: 1.1 GB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365405 entries, 2014-09-29 05:35:00 to 2018-03-20 23:55:00
Columns: 213 entries, Open to up_down
dtypes: float64(204), int64(9)
memory usage: 596.6 MB
None


In [40]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [41]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709314 entries, 0 to 709313
Columns: 213 entries, Open to up_down
dtypes: float64(204), int64(9)
memory usage: 1.1 GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365405 entries, 0 to 365404
Columns: 213 entries, Open to up_down
dtypes: float64(204), int64(9)
memory usage: 593.8 MB
None


In [20]:
train.to_csv('split_by_time/Train_full.csv')

In [21]:
cut_at = int(np.floor(len(train)*.93))
train_small = train.iloc[cut_at:,:]
train_small = train_small.reset_index(drop=True)

train_small.to_csv('split_by_time/Train_small.csv')
del train, train_small

In [44]:
test_lable = {'num': np.arange(len(test)), 'label': test['up_down']}
test_lable = pd.DataFrame(test_lable,columns=['label','num'])
test_lable = test_lable.set_index(['num'])

test_features = test.drop(columns = ['up_down'])

test_lable.to_csv('split_by_time/Test_label.csv')
test_features.to_csv('split_by_time/Test_features.csv')

In [45]:
cut_at = int(np.floor(len(test)*.03))
test_small = test.iloc[:cut_at,:]
test_small = test_small.reset_index(drop=True)

test_small_lable = {'num': np.arange(len(test_small)), 'label': test_small['up_down']}
test_small_lable = pd.DataFrame(test_small_lable,columns=['label','num'])
test_small_lable = test_small_lable.set_index(['num'])

test_small_features = test_small.drop(columns = ['up_down'])

test_small_lable.to_csv('split_by_time/Test_small_label.csv')
test_small_features.to_csv('split_by_time/Test_small_features.csv')


In [46]:
del test_lable, test_features, test_small_lable, test_small_features, test, test_small

### Split Sample Randomly

In [47]:
print(data.info())
data = data.reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1074719 entries, 2008-01-01 08:05:00 to 2018-03-20 23:55:00
Columns: 213 entries, Open to up_down
dtypes: float64(204), int64(9)
memory usage: 1.7 GB
None


In [48]:
data.head(10)

,Open,High,Low,Close,Volume,body,upper_tail,lower_tail,SMA_50,SMA_20,...,lag_return_92,return_93,lag_return_93,return_94,lag_return_94,return_95,lag_return_95,return_96,lag_return_96,up_down
0,1.45865,1.45885,1.45856,1.45873,5004.3375,0.8,1.2,0.9,1.459251,1.459150,...,-0.000089,-0.000528,0.000027,-0.000500,-0.000103,-0.000603,0.000226,-0.000377,0.000933,0
1,1.45883,1.45883,1.45847,1.45863,4649.9000,-2.0,0.0,1.6,1.459236,1.459130,...,0.000103,-0.000507,-0.000089,-0.000596,0.000027,-0.000569,-0.000103,-0.000671,0.000226,1
2,1.45863,1.45896,1.45845,1.45886,5283.9500,2.3,1.0,1.8,1.459217,1.459097,...,0.000048,-0.000452,0.000103,-0.000349,-0.000089,-0.000439,0.000027,-0.000411,-0.000103,1
3,1.45865,1.45905,1.45842,1.45895,5370.3375,3.0,1.0,2.3,1.459204,1.459044,...,-0.000644,-0.000438,0.000048,-0.000391,0.000103,-0.000288,-0.000089,-0.000377,0.000027,0
4,1.45902,1.45905,1.45857,1.45887,3738.9375,-1.5,0.3,3.0,1.459187,1.458988,...,-0.000048,0.000151,-0.000644,-0.000493,0.000048,-0.000445,0.000103,-0.000343,-0.000089,1
5,1.45883,1.45915,1.45852,1.45913,5427.8375,3.0,0.2,3.1,1.459186,1.458943,...,-0.000514,0.000377,-0.000048,0.000329,-0.000644,-0.000315,0.000048,-0.000267,0.000103,0
6,1.45905,1.45922,1.45881,1.45887,4256.5000,-1.8,1.7,0.6,1.459168,1.458883,...,0.000014,0.000713,-0.000514,0.000199,-0.000048,0.000151,-0.000644,-0.000493,0.000048,0
7,1.45892,1.45899,1.45848,1.45883,4874.0500,-0.9,0.7,3.5,1.459154,1.458886,...,0.000055,0.000672,0.000014,0.000686,-0.000514,0.000171,-0.000048,0.000123,-0.000644,0
8,1.45893,1.45906,1.45861,1.45862,4751.0469,-3.1,1.3,0.1,1.459132,1.458883,...,-0.000117,0.000473,0.000055,0.000528,0.000014,0.000542,-0.000514,0.000027,-0.000048,1
9,1.45885,1.45895,1.45848,1.45880,4313.4000,-0.5,1.0,3.2,1.459120,1.458884,...,0.000027,0.000713,-0.000117,0.000597,0.000055,0.000652,0.000014,0.000665,-0.000514,0


In [49]:
from sklearn.model_selection import train_test_split

y = data['up_down']
X = data.drop(columns = ['up_down'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 33)
del y,X

In [50]:
X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(
    X_train, y_train, test_size = 0.03, random_state = 22)

X_test_small.loc[:,'up_down'] = y_test_small

X_test_small.to_csv('random_split/Train_small.csv')
del X_train_small, X_test_small, y_train_small, y_test_small 

/home/jacob/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/jacob/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [51]:
X_train.loc[:,'up_down'] = y_train
X_train.to_csv('random_split/Train.csv')
del X_train,y_train

In [52]:
X_test = X_test.reset_index(drop=True)
X_test.to_csv('random_split/Test_features.csv')


test_lable = {'num': np.arange(len(y_test)), 'label': y_test}
test_lable = pd.DataFrame(test_lable,columns=['label','num'])
test_lable = test_lable.set_index(['num'])

test_lable.to_csv('random_split/Test_label.csv')

In [53]:
X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(
    X_test, y_test, test_size = 0.07, random_state = 22)

X_test_small = X_test_small.reset_index(drop=True)
X_test_small.to_csv('random_split/Test_small_features.csv')

test_small_lable = {'num': np.arange(len(y_test_small)), 'label': y_test_small}
test_small_lable = pd.DataFrame(test_small_lable,columns=['label','num'])
test_small_lable = test_small_lable.set_index(['num'])

test_small_lable.to_csv('random_split/Test_small_label.csv')

del X_train_small, X_test_small, y_train_small, y_test_small, test_small_lable, test_lable
del X_test, y_test